# Охват иммунизацией детей в возрасте 1 года против гепатита В (ГепВ3) (%)

In [15]:
import pandas as pd

from helpers import get_data_with_full_locations_and_years

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке:

https://www.who.int/data/gho/data/indicators/indicator-details/GHO/hepatitis-b-(hepb3)-immunization-coverage-among-1-year-olds-(-)

In [16]:
# Загрузим таблицу
data = pd.read_csv('../data/who_hepatitis_b_immunization.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,WHS4_117,Hepatitis B (HepB3) immunization coverage amon...,numeric,WPR,Western Pacific,Country,PNG,Papua New Guinea,Year,2020,...,NaN,NaN,NaN,NaN,NaN,40,NaN,NaN,EN,2024-07-11T21:00:00.000Z
1,WHS4_117,Hepatitis B (HepB3) immunization coverage amon...,numeric,AFR,Africa,Country,CAF,Central African Republic,Year,2020,...,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,EN,2024-07-11T21:00:00.000Z


Описание нужных нам столбцов.

- `ParentLocationCode` - код региона
- `SpatialDimValueCode` - код страны
- `Period` - год
- `FactValueNumeric` - охват иммунизацией детей в возрасте 1 года против гепатита В (ГепВ3) (%)

Оставим только их.

In [17]:
data = data[[
    'ParentLocationCode', 'SpatialDimValueCode', 'Period', 'FactValueNumeric',
]]

# Заменим название столбца FactValueNumeric на HepatitisBImmunization
data.rename(
    columns={'FactValueNumeric': 'HepatitisBImmunization'},
    inplace=True,
)

data.head()

,ParentLocationCode,SpatialDimValueCode,Period,HepatitisBImmunization
0,WPR,PNG,2020,40
1,AFR,CAF,2020,42
2,EMR,SOM,2020,42
3,AFR,GIN,2020,47
4,AFR,AGO,2020,51


В таблице нет информации о следующих странах за все годы:

- Венгрия (*Hungary*), код страны *HUN*, код региона *EUR*
- Финляндия (*Finland*), код страны *FIN*, код региона *EUR*
- Словения (*Slovenia*), код страны *SVN*, код региона *EUR*
- Исландия (*Iceland*), код страны *ISL*, код региона *EUR*
- Дания (*Denmark*), код страны *DNK*, код региона *EUR*

Добавим строки с пустыми данными об этих странах для 2000 г.

In [18]:
# Создадим пустую таблицу с такими же колонками, как у исходной
new_data = pd.DataFrame(columns=list(data.columns))

# Заполним данные отсутствующих стран только для 2000 г.
new_data['SpatialDimValueCode'] = ['HUN', 'FIN', 'SVN', 'ISL', 'DNK']
new_data['ParentLocationCode'] = 'EUR'
new_data['Period'] = 2000

# Добавим их в исходную таблицу
data = pd.concat([
    data,
    new_data,
], ignore_index=True)

Также в таблице представлены данные для стран не за все года.\
Это связано с тем, что данные о вакцинации в некоторых странах начинали фиксироваться позже 2000г.\
Дополним таблицу строками с отсутствующими годами, чтобы явно увидеть пропуски.

In [19]:
data = get_data_with_full_locations_and_years(
    data,
    {
        'region_code': 'ParentLocationCode',
        'location_code': 'SpatialDimValueCode',
        'year': 'Period',
    }
)

# Посмотрим на пропуски
data.isna().sum()

ParentLocationCode          0
SpatialDimValueCode         0
Period                      0
HepatitisBImmunization    601
dtype: int64

Заполним пропуски в пропущенные года средним по региону.

In [20]:
data['HepatitisBImmunization'] = \
    data.groupby(['ParentLocationCode', 'Period'])['HepatitisBImmunization'] \
    .transform(lambda x: x.fillna(x.mean()));

# Убедимся, что пропусков нет
data.isna().sum().sum()

0

In [21]:
# Удалим столбцы, которые больше не нужны
data.drop(columns=['ParentLocationCode'], inplace=True)

In [22]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/who_hepatitis_b_immunization_prepared.csv', index=False)